In [2]:
# Import modules
import pandas as pd
import json
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:

subtask1 = pd.read_csv('../res/raw/subtask1.csv')
subtask2 = pd.read_csv('../res/raw/subtask2.csv')
subtask2_n_3 = json.load(open('../res/raw/subtask2_n_3.json'))
subtask3 = json.load(open('../res/raw/subtask3.json'))

### Collect data from subtask1

In [1]:
# subtask1.loc[(subtask1['severe_toxic']) == 1 & (subtask1['threat'] == 0)]
# subtask1.loc[(subtask1['severe_toxic']) == 1 & (subtask1['threat'] == 1)]
'''
 Get all hate speech rows
 QUESTION: How did you come up with such conditions?
 ANSWER: All posts that have a toxic value of 1 are considered 
 somewhat hateful however they are not yet fully considered as hate speech.
 Note: Category Columns (toxic, severe_toxic, obscene, etc.)
 Given a row, if and when some of it's other category columns have a value of 1
 then it is highly likely that the post is a hate speech. This conclusion
 came from manually looking at random rows that has a toxic value of 1 with
 any category column that has a value of 1 as well. 
'''
subtask1_df = (subtask1.loc[(subtask1['toxic'] == 1)  &
                   ((subtask1['severe_toxic'] == 1) |
                   (subtask1['obscene'] == 1) |
                   (subtask1['threat'] == 1) |
                   (subtask1['insult'] == 1) |
                   (subtask1['identity_hate'] == 1))])
       


# print(subtask1_df.shape)
# test_subtask1_df = subtask1_df.loc[subtask1_df['identity_hate']==1]
# print(test_subtask1_df.shape)
# display(test_subtask1_df['comment_text'].apply(lambda x: ''.join(x.strip().split('\n'))))


NameError: name 'subtask1' is not defined

### Collect data from subtask2

### Collect data from subtask2_n_3

### Collect data from subtask3